In [3]:
import numpy as np
import cv2
import mediapipe as mp
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder
import time
from collections import Counter

# Load the trained CNN model
model = load_model("C:\\Users\\Akshaj Seerpu\\Desktop\\College\\RAPID\\sign_language_model_2.keras")

# Load label encoder
label_encoder = LabelEncoder()
label_encoder.classes_ = np.load("C:\\Users\\Akshaj Seerpu\\Desktop\\College\\RAPID\\label_encoder_classes.npy", allow_pickle=True)

# MediaPipe Hands setup
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.7)

# Initialize variables for sentence formation
sentence = []
predictions = []
last_detected_time = time.time()
last_sign_time = time.time()

# Function to predict sign language word from keypoints
def predict_sign_language(keypoints):
    keypoints = np.array(keypoints)
    keypoints = keypoints.reshape(1, len(keypoints), 1)
    prediction = model.predict(keypoints)
    label_index = np.argmax(prediction)
    predicted_label = label_encoder.inverse_transform([label_index])[0]
    return predicted_label

# Function to process each frame from the camera feed
def process_frame(frame, sentence, predictions, last_sign_time, last_detected_time):
    # Convert the BGR image to RGB
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # Flip the image horizontally for a later selfie-view display
    image_rgb = cv2.flip(image_rgb, 1)
    # To improve performance, optionally mark the image as not writeable to pass by reference
    image_rgb.flags.writeable = False
    # Process the image
    results = hands.process(image_rgb)
    # Draw the hand landmarks on the image
    image_rgb.flags.writeable = True
    image_rgb = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)
    
    current_time = time.time()
    
    if results.multi_hand_landmarks:
        last_detected_time = current_time
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                image_rgb, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            # Extract keypoints
            keypoints = []
            for landmark in hand_landmarks.landmark:
                keypoints.append(landmark.x)
                keypoints.append(landmark.y)
                keypoints.append(landmark.z if landmark.z else 0)  # Some landmarks might not have z
            # Predict sign language word
            predicted_label = predict_sign_language(keypoints)
            predictions.append(predicted_label)
            
            # Display predicted label
            cv2.putText(image_rgb, f'Predicted: {predicted_label}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            
            # If 3 seconds have passed since the last sign was added, add the most frequent sign
            if current_time - last_sign_time > 3:
                most_common_prediction = Counter(predictions).most_common(1)[0][0]
                sentence.append(most_common_prediction)
                predictions = []
                last_sign_time = current_time
                
    # Display the sentence and timer on the screen
    cv2.putText(image_rgb, f'Sentence: {" ".join(sentence)}', (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.putText(image_rgb, f'Word Timer: {int(3 - (current_time - last_sign_time))}s', (10, 110), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.putText(image_rgb, f'Sentence Timer: {int(5 - (current_time - last_detected_time))}s', (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    
    return image_rgb, sentence, predictions, last_sign_time, last_detected_time

# Main function to capture video from camera
def main():
    cap = cv2.VideoCapture(0)
    sentence = []
    predictions = []
    last_sign_time = time.time()
    last_detected_time = time.time()
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Ignoring empty camera frame.")
            continue
        
        # Process each frame
        processed_frame, sentence, predictions, last_sign_time, last_detected_time = process_frame(frame, sentence, predictions, last_sign_time, last_detected_time)
        
        # Display the resulting frame
        cv2.imshow('Sign Language Recognition', processed_frame)
        
        # Exit the loop if the 'q' key is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Write the sentence to the text file upon exiting
    with open("sentence.txt", "w") as f:
        f.write(" ".join(sentence))

    # Release the capture object and destroy all windows
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step

C:\Users\Akshaj Seerpu\AppData\Roaming\Python\Python312\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step


C:\Users\Akshaj Seerpu\AppData\Roaming\Python\Python312\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\Akshaj Seerpu\AppData\Roaming\Python\Python312\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step

...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step


C:\Users\Akshaj Seerpu\AppData\Roaming\Python\Python312\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\Akshaj Seerpu\AppData\Roaming\Python\Python312\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step


C:\Users\Akshaj Seerpu\AppData\Roaming\Python\Python312\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\Akshaj Seerpu\AppData\Roaming\Python\Python312\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
